In [ ]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.6 MB/s eta 0:00:00


In [26]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph import message
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, prompt
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage

import operator
from typing import Literal, TypedDict, Annotated
from pydantic import Field, BaseModel
from pprint import pprint
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()

key = userdata.get('OPENAI_API_KEY')

#llm = ChatOpenAI(model = 'gpt-5.1')
llm = ChatOpenAI(api_key = key)

class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

config1 = {"configurable": {"thread_id": "1"}}
response = workflow.invoke({'topic':'pizza'}, config=config1)

In [27]:
response

{'topic': 'pizza',
 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle all the stress!",
 'explanation': 'This joke is a play on words, using the idea of a pizza going to therapy for emotional stress. The joke plays on the common phrase "having too many toppings," which in this case is used to depict the pizza being overwhelmed by the stress of carrying too many toppings. The humor comes from the absurdity of a pizza seeking therapy and the idea that toppings could cause emotional distress.'}

In [28]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle all the stress!", 'explanation': 'This joke is a play on words, using the idea of a pizza going to therapy for emotional stress. The joke plays on the common phrase "having too many toppings," which in this case is used to depict the pizza being overwhelmed by the stress of carrying too many toppings. The humor comes from the absurdity of a pizza seeking therapy and the idea that toppings could cause emotional distress.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5c6-e4ba-6f72-8002-5f942d803ebd'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-28T13:16:20.799642+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5c6-d7b1-61d3-8001-5b6804ed23a9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 

In [30]:
#Lets get the State at generate_joke. Get its checkpoint_id from above
# 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'

list(workflow.get_state_history({"configurable": {"thread_id": "1", 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'}}))

[StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-28T13:16:18.330886+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5c6-cd2a-6796-bfff-7c329e0262ca'}}, tasks=(PregelTask(id='c647894f-8401-d6af-0260-f53267d67ced', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle all the stress!"}),), interrupts=())]

In [31]:
#Lets resume from there
workflow.invoke(None, {"configurable": {"thread_id": "1", 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'}})

{'topic': 'pizza',
 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!",
 'explanation': "This joke plays on the idea of a pizza going to therapy by personifying the pizza and giving it human characteristics. The punchline reveals that the pizza had too many toppings, which created too much pressure on it, causing it to seek therapy. It's a light-hearted way of poking fun at the idea of someone seeking therapy for their stress or anxiety, but in this case, it's a pizza with too many toppings."}

In [ ]:
#You can see here and the before reponse, the jokes are different.
#This feature we can see in Chatgpt Edit chat feature, where in middle of conversation,
#we can edit a presvious chat and a new chat gets resume.

In [32]:
pprint(list(workflow.get_state_history(config1)))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': "This joke plays on the idea of a pizza going to therapy by personifying the pizza and giving it human characteristics. The punchline reveals that the pizza had too many toppings, which created too much pressure on it, causing it to seek therapy. It's a light-hearted way of poking fun at the idea of someone seeking therapy for their stress or anxiety, but in this case, it's a pizza with too many toppings."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5ca-9fc8-69ef-8002-ab7027aef22c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-28T13:18:00.944239+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5ca-8e55-6591-8001-f050691a83c9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'t

In [ ]:
## Update State

In [33]:
# We can also change any state value and execute it from there. Just like Chatgpt Edit chat feature
# Lets change the topic from Pizza to Samosa
# Get checkpointid for first time topic: pizza i.e., 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'
list(workflow.get_state_history({"configurable": {"thread_id": "1", 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'}}))

[StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-28T13:16:18.330886+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5c6-cd2a-6796-bfff-7c329e0262ca'}}, tasks=(PregelTask(id='c647894f-8401-d6af-0260-f53267d67ced', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle all the stress!"}),), interrupts=())]

In [35]:
workflow.update_state({"configurable": {"thread_id": "1", 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b', 'checkpoint_ns':''}}, {'topic': 'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0cc5ce-2632-6fb6-8001-6b5ad7d52bc2'}}

In [36]:
pprint(list(workflow.get_state_history(config1))) #We can see a new state got added at the top

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5ce-2632-6fb6-8001-6b5ad7d52bc2'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-11-28T13:19:35.569333+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5c6-cd2f-6b4d-8000-e93c6a18187b'}}, tasks=(PregelTask(id='ff596cce-af1b-4627-d7e5-c86c7a3515d0', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': "This joke plays on the idea of a pizza going to therapy by personifying the pizza and giving it human characteristics. The punchline reveals that the pizza had too many toppings, which created too much pres

In [37]:
#Now to execute from Samosa, get the Checkpointid for the new edit i.,e the top most Samosa one
#'checkpoint_id': '1f0cc5ce-2632-6fb6-8001-6b5ad7d52bc2'
workflow.invoke(None, {"configurable": {"thread_id": "1", 'checkpoint_id': '1f0cc5ce-2632-6fb6-8001-6b5ad7d52bc2'}})


{'topic': 'samosa',
 'joke': 'Why did the samosa go to school?\n\nTo become a little more well-rounded!',
 'explanation': 'This joke plays on the double meaning of the phrase "well-rounded." In one sense, it can mean having a well-developed and diverse range of skills or knowledge, which is something that attending school can help achieve. In another sense, it can mean physically round in shape, like a samosa. So the joke is saying that the samosa went to school to become more well-developed and diverse, but also to become more physically round in shape.'}

In [38]:
pprint(list(workflow.get_state_history(config1))) #We can see a new state got added at the top

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to school?\n\nTo become a little more well-rounded!', 'explanation': 'This joke plays on the double meaning of the phrase "well-rounded." In one sense, it can mean having a well-developed and diverse range of skills or knowledge, which is something that attending school can help achieve. In another sense, it can mean physically round in shape, like a samosa. So the joke is saying that the samosa went to school to become more well-developed and diverse, but also to become more physically round in shape.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5d5-2416-6883-8003-26bb710d501b'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-11-28T13:22:43.252823+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cc5d5-1bbf-664c-8002-d3d0ca30969e'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'to